In [4]:
#Load dataset
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston

data , housing_prices  = load_boston(return_X_y=True)


In [5]:
print (data.shape)

(506, 13)


In [11]:
# We need to define w_Tilde, our X is our data augmented by one and Y is our housing prices 
n = len(data)
m = len(housing_prices)
def least_squared(housing_prices,data):
    
    features = np.column_stack((data,np.ones((506,1))))
    Y = housing_prices.reshape(506,1)
    #W_Tilde is defined as (X^TX)^-1*X^T*Y
    W_Tilde = np.linalg.inv(features.T @ features) @ features.T @ Y
   # print (W_Tilde.shape)
    #Now we need to define Yi as Xi.T @ w_Tilde
    Y_hat = features @ W_Tilde
   # print (Y_hat.shape)
   # print (Y.shape)
    sum = 0
    #We then calulate the error by running a for loop of (Y-Y_hat)^2
    for i in range(506):
        sum = sum +((Y[i]-Y_hat[i])**2)
    print("Average prediction error" , sum/506) #Average prediction error
    return sum
least_squared(housing_prices,data)

Average prediction error [21.89777922]


array([11080.27628415])

In [26]:
# We now need to implement the 5-fold cross validation
# We will need to do 5k-fold cross validation
#Essentially what K-fold is, is running Least squared in a for loop 5 times using a training sets
#We have 506 samples, so we need to have 506/5 ~= 101 samples per run
def Cross_Validation(housing_prices,data):
    #First before we do k-fold we need to shuffle the data
    joined = np.concatenate((data,Y),axis = 1)
    np.random.shuffle(joined)
    shuffle_data = joined.T[0:13].T # We have 13 variables
    shuffle_housing_prices=joined[:,13]
    error_array = []
    #Next we do the k-fold where k=5
    for k in range (0,5):
        if(k==4):
            # make indexes that will be the number of our samples
            index1 = 404
            index2 = 506
        else: 
            index1 = (k*101)
            index2 = index1+101
        # We know announce the test set and the training set
        test_data = shuffle_data[index1:index2]
        test_prices = shuffle_housing_prices[index1:index2]
        training_data = np.concatenate((shuffle_data[0:index1],shuffle_data[index2:506]))
        training_prices = np.concatenate((shuffle_housing_prices[0:index1],shuffle_housing_prices[index2:506]))
            
        W = least_squared(training_prices,training_data)
        augmented_data = np.concatenate((data,np.ones((data.shape[0],1))),axis=1)
        output = augmented_data @ W
        Labels = np.reshape(test_prices,(output.shape[0],1))
        difference = np.linalg.norm(output-labels,2)
        error_array.append((difference**2)/output.shape[0])
            
        # Plot the data
        fig = plt.figure()
        ax = fig.add_subplot(111)
        x = list(*zip(output.tolist()))
        y = list(*zip(labels.tolist()))
        ax.scatter(y[0],x[0],c='k')
        ax.plot(np.arange(output.shape[0]),np.arange(output.shape[0]), 'r')
        plt.title('K-Fold Cross Validation, k='+str(k+1));
        ax.set_xlabel('Measured')
        ax.set_ylabel('Predicted')
        
    return error_array
Result = Cross_Validation(housing_prices,data)
print("The error is ", Result)
print("The average from the CV is " , mean(output))
    
    

ValueError: all the input array dimensions except for the concatenation axis must match exactly